# **Microsoft Vision classification example**

This example shows a simple way to use Microsoft vision with PyTorch for transfer learning and feature extraction from the Microsoft Vision model.

This shows an example to plug-in a fully connected Neural network on top of the vision model which provides features from the data..

Reading the necessary imports.

In [ ]:
import time
import torch
import numpy as np
import torch.nn as nn
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from progressbar import progressbar

##  Install the Microsoft Vision

In [ ]:
!pip install microsoftvision

import microsoftvision

     |████████████████████████████████| 112kB 9.5MB/s 
     |████████████████████████████████| 348kB 14.5MB/s 
     |████████████████████████████████| 133kB 17.9MB/s 
     |████████████████████████████████| 3.2MB 18.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


## Preprocess the Input Images

Microsoft Vision model is using images in BGR format, hence the swapping of image channels at the end of preprocessing

In [ ]:
class Preprocess:
    def __init__(self):
        self.preprocess = transforms.Compose([
                                           transforms.Resize(224),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])])

    def __call__(self, x):
        return self.preprocess(x)[[2,1,0],:,:]

Import the CIFAR-10 dataset with the division to train and test sets. This can be replaced with any dataset without any changes to the rest of the code.

Even this can be replaced with PyTorch with torchvision.datasets.ImageFolder

A generic data loader where the images are arranged in this way:

>root/dog/xxx.png<br>
> Indented block<br>
> root/dog/xxy.png<br>
> Indented block<br>
> Indented block<br>
>root/dog/[...]/xxz.png<br>
>root/cat/123.png<br>
>root/cat/nsdf3.png<br>
>root/cat/[...]/asd932_.png<br>

In [ ]:
train_dataset = CIFAR10('path', download=True, train=True, transform=Preprocess())
test_dataset = CIFAR10('path', download=True, train=False, transform=Preprocess())


Extracting path/cifar-10-python.tar.gz to path
Files already downloaded and verified


## Loading Microsoft Vision pretrained model

In [ ]:
model = microsoftvision.models.resnet50(pretrained=True)

Loading Microsoft Vision pretrained model


  0%|          | 0/23 [00:00<?, ?MB/s]

Model size: 89 MB


100%|██████████| 23/23 [00:50<00:00,  2.18s/MB]

Model saved to MicrosoftVision.ResNet50.tar


In [ ]:
# using GPU for speed-up computation
model.eval()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Feature Extarction

In [ ]:
def get_features(dataset, model):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in progressbar(DataLoader(dataset, batch_size=128, num_workers=8)):
            images = images.cuda()
            labels = labels.cuda()
            features = model(images)

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu(), torch.cat(all_labels).cpu()

In [ ]:
train_features, train_labels = get_features(train_dataset, model)
test_features, test_labels = get_features(test_dataset, model)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100% (391 of 391) |######################| Elapsed Time: 0:05:00 Time:  0:05:00
100% (79 of 79) |########################| Elapsed Time: 0:01:00 Time:  0:01:00


In [ ]:
# train features preprocessing

train_labels = train_labels.to(dtype=torch.float)
train = TensorDataset( train_features, train_labels)

# Create a data loader from the features
train_loader = DataLoader(train, batch_size= 30,shuffle=True)

In [ ]:
# test features preprocessing

test_labels = test_labels.to(dtype=torch.float)
test = TensorDataset( Tensor(test_features), Tensor(test_labels))

# Create a data loader from the features
test_loader = DataLoader(test, batch_size= 10)

## Custom Fully Connected Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(2048, 1024)
        self.out = nn.Linear(1024, 10)
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.out(x)
        return x

#instantiate our Neural Network class and moving it to the GPU
network =  Network().cuda()

In [ ]:

#initialize Cross Entropy loss function
criterion = nn.CrossEntropyLoss()


# set Stochastic Gradient Descent as optimizer function
optimizer = optim.RMSprop(network.parameters(), lr=0.0000005) # Optimizer



In [ ]:
dataset_sizes = {'train':len(train_labels),'test':len(test_labels)}

In [ ]:
device = 'cuda'
def train_model(model, criterion, optimizer, num_epochs=20):
    since = time.time()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode


        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in progressbar(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.to(dtype=torch.long)

            # zero the parameter gradients
            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']

        print(' Loss: {:.4f} Acc: {:.4f}'.format(
             epoch_loss, epoch_acc))



    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    return model

In [ ]:
model_ft = train_model(network, criterion,optimizer,num_epochs=10)

  2% (43 of 1667) |                      | Elapsed Time: 0:00:00 ETA:   0:00:05

Epoch 0/9
----------


100% (1667 of 1667) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
  3% (64 of 1667) |                      | Elapsed Time: 0:00:00 ETA:   0:00:04

 Loss: 1.5531 Acc: 0.7010
Epoch 1/9
----------


100% (1667 of 1667) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
  3% (64 of 1667) |                      | Elapsed Time: 0:00:00 ETA:   0:00:04

...

 Loss: 0.2323 Acc: 0.9271
Epoch 8/9
----------


100% (1667 of 1667) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
  3% (64 of 1667) |                      | Elapsed Time: 0:00:00 ETA:   0:00:04

 Loss: 0.2242 Acc: 0.9285
Epoch 9/9
----------


100% (1667 of 1667) |####################| Elapsed Time: 0:00:04 Time:  0:00:04


 Loss: 0.2177 Acc: 0.9298
Training complete in 0m 47s


## Testing the model with Test Dataset

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in progressbar(test_loader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

100% (1000 of 1000) |####################| Elapsed Time: 0:00:01 Time:  0:00:01


Accuracy of the network on the test images: 92 %
